In [43]:
import pandas as pd 
data =  pd.read_csv( 'culture.csv' , encoding='utf-8') # dataframe  

print(data.iloc[1:2,0])

# for i in data :
#     print(i)

1    "머리 짧다고 때리다니"…숏컷 인증샷으로 '여혐 범죄' 맞서는 여성들
Name: title, dtype: object


In [44]:
data

,title,reporter,content,category,date
0,"[날씨] 낮에도 찬 바람 계속...내일 아침, 오늘보다 더 추워",익명,\n\t\t\t볼에 닿는 공기가 여전히 차갑습니다.현재 서울 기온은 7.7도로 어제...,culture,2023.11.07. 오후 1:35
1,"""머리 짧다고 때리다니""…숏컷 인증샷으로 '여혐 범죄' 맞서는 여성들",최성규 기자,\n\n\n\n\n사회관계망서비스(SNS)를 중심으로 '여성 숏컷 캠페인'이 펼쳐지...,culture,2023.11.07. 오후 1:32
2,“또 사건 만들려고 왔니?” 대전 한 초교 앞 붙은 현수막,최예슬 기자,\n\n\n\n\n사진=온라인 커뮤니티 갈무리악성 민원을 지속해 교사를 극단적 선택...,culture,2023.11.07. 오후 1:32
3,"""머리 짧다고 때리다니""…숏컷 인증샷으로 '여혐 범죄' 맞서는 여성들",최성규 기자,\n\n\n\n\n사회관계망서비스(SNS)를 중심으로 '여성 숏컷 캠페인'이 펼쳐지...,culture,2023.11.07. 오후 1:32
4,"[단독] 쿠팡이츠, 야식 배달 시간 더 늘린다…24시간도 초읽기",류선우 기자,\n서울·경기·인천서 새벽 3시까지 영업…'24시간' 영업도 추진\n\n\n\n쿠팡...,culture,2023.11.07. 오후 1:07
...,...,...,...,...,...
115,"컨테이너 떨어지고, 철문에 부딪히고…대전·충남 강풍피해 속출(종합)",이주형 기자,"\n전지역 강풍특보…대전 5건, 세종 6건, 충남45건 피해신고 접수\n\n\n\n...",culture,2023.11.06. 오후 5:48
116,[날씨] 전국 대부분 강풍특보 발효…밤사이 10~15도 '뚝',익명,\n\t\t\t종일 바람이 강하게 불고 있습니다.현재 전국 대부분 지역에는 강풍특보...,culture,2023.11.06. 오후 5:46
117,[날씨] 전국 대부분 강풍특보 발효…밤사이 10~15도 '뚝',익명,\n\t\t\t종일 바람이 강하게 불고 있습니다.현재 전국 대부분 지역에는 강풍특보...,culture,2023.11.06. 오후 5:46
118,폐암은 죽을 병?…생존율 높이는 방법은?,임태균 기자,"\n전체 암 중 사망률 1위, 5년 생존율은 30% 수준85%는 흡연과 연관…시작 ...",culture,2023.11.06. 오후 5:45


In [7]:
import mysql.connector
import csv
from datetime import datetime
import re

# MySQL 데이터베이스 연결 설정
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="0000",
    database="news"
)

cursor = conn.cursor()

# CSV 파일 열기
with open('world.csv', 'r', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    # 첫 번째 행을 건너뛰고 나머지 행 처리
    next(csvreader, None)  # 첫 번째 행을 건너뛰기
    
    # CSV 파일 읽기
    for row in csvreader: 
        # 데이터 파싱 및 DATETIME 형식으로 변환
        write_date_str = row[4]

        # 정규식을 사용하여 오전/오후 정보 분리
        match = re.match(r'(\d{4}.\d{2}.\d{2}.)\s(오전|오후)\s(\d{1,2}:\d{2})', write_date_str)
        if match:
            date_str, period, time_str = match.groups()
            if period == '오후':
                time_obj = datetime.strptime(date_str + time_str, '%Y.%m.%d.%I:%M')  # %p는 오전/오후를 나타냄
                time_obj = time_obj.replace(hour=time_obj.hour + 12)  # 오후인 경우 12를 더하여 24시간 형식으로 변경
            else:
                time_obj = datetime.strptime(date_str + time_str, '%Y.%m.%d.%I:%M')
    
        # DATETIME 형식으로 데이터베이스에 삽입
        cursor.execute("INSERT INTO article (title, reporter_name, content, category, write_date) VALUES (%s, %s, %s, %s, %s)",
                       (row[0], row[1], row[2], row[3], time_obj))

# 데이터베이스에 변경사항 저장
conn.commit()

# 데이터베이스 연결 종료
conn.close()
